In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Đọc dữ liệu từ file CSV
file_path = '/content/drive/My Drive/Datasets/IMDB Dataset.csv'
data = pd.read_csv(file_path)

# Tiền xử lý dữ liệu: Chuyển đổi nhãn thành 0 (negative) và 1 (positive)
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

# Tạo tokenizer và từ điển
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

# Xây dựng từ điển cho các từ trong dataset
vocab = build_vocab_from_iterator(yield_tokens(data['review']), specials=["<PAD>", "<UNK>"])
vocab.set_default_index(vocab["<UNK>"])

# Chuyển đổi các review thành chỉ số
def preprocess_data(texts, vocab, tokenizer, max_len=200):
    encoded_texts = []
    for text in texts:
        tokens = tokenizer(text)
        encoded_text = [vocab[token] for token in tokens][:max_len]
        padded_text = encoded_text + [vocab["<PAD>"]] * (max_len - len(encoded_text))
        encoded_texts.append(torch.tensor(padded_text))
    return pad_sequence(encoded_texts, batch_first=True)

# Tiền xử lý dữ liệu văn bản
X = preprocess_data(data['review'], vocab, tokenizer)
y = torch.tensor(data['sentiment'].values)

# Chia dữ liệu thành train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuyển dữ liệu thành DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [23]:
import torch.nn as nn
import torch.optim as optim

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_classes, kernel_sizes, num_filters, dropout_rate=0.5):
        super(TextCNN, self).__init__()

        # Lớp embedding
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Các lớp Conv1d cho nhiều kích thước kernel khác nhau
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_size, out_channels=num_filters, kernel_size=k)
            for k in kernel_sizes
        ])

        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        # Lớp fully connected
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

    def forward(self, x):
        # Lớp embedding
        x = self.embedding(x)  # Chuyển từ index thành embedding

        # Đổi chiều để phù hợp với Conv1D (batch_size, embed_size, seq_length)
        x = x.transpose(1, 2)  # (batch_size, embed_size, seq_length)

        # Áp dụng các lớp Conv1D
        conv_results = [torch.relu(conv(x)) for conv in self.convs]

        # Max pooling trên chiều chiều dài chuỗi (seq_length)
        pooled_results = [torch.max(conv_result, dim=2)[0] for conv_result in conv_results]

        # Kết hợp các kết quả pooling lại với nhau
        cat = torch.cat(pooled_results, dim=1)

        # Áp dụng Dropout
        cat = self.dropout(cat)

        # Lớp fully connected
        output = self.fc(cat)

        return output




# Các tham số mô hình
vocab_size = len(vocab)  # Từ điển vocab đã xây dựng trước đó
embed_size = 128
num_classes = 2  # Số lớp (positive, negative)
kernel_sizes = [3, 4, 5]  # Các kích thước kernel khác nhau
num_filters = 100  # Số lượng filter
dropout_rate = 0.5 # Tỷ lệ dropout

# Khởi tạo mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextCNN(vocab_size, embed_size, num_classes, kernel_sizes, num_filters, dropout_rate).to(device)
print(device)

cuda


In [24]:
# Thiết lập các tham số huấn luyện
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # weight_decay = L2 regularization
loss_fn = nn.CrossEntropyLoss()

# Huấn luyện mô hình
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in train_loader:
        inputs, labels = batch
        # Di chuyển inputs và labels sang cùng device với model
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    # Tính accuracy
    accuracy = 100 * correct_predictions / total_predictions
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}, Accuracy: {accuracy}%")

Epoch 1/15, Loss: 0.5870006852388382, Accuracy: 68.96%
Epoch 2/15, Loss: 0.43018938430547715, Accuracy: 80.1%
Epoch 3/15, Loss: 0.34929062739014627, Accuracy: 84.57%
Epoch 4/15, Loss: 0.28903667719066145, Accuracy: 87.9475%
Epoch 5/15, Loss: 0.24130050117075444, Accuracy: 90.21%
Epoch 6/15, Loss: 0.19717907936424017, Accuracy: 92.2475%
Epoch 7/15, Loss: 0.16268429799154402, Accuracy: 93.685%
Epoch 8/15, Loss: 0.12798389133363963, Accuracy: 95.2025%
Epoch 9/15, Loss: 0.10987095902599395, Accuracy: 95.8625%
Epoch 10/15, Loss: 0.09276870388947427, Accuracy: 96.5075%
Epoch 11/15, Loss: 0.08502690050564707, Accuracy: 96.7975%
Epoch 12/15, Loss: 0.07831132271802053, Accuracy: 97.095%
Epoch 13/15, Loss: 0.06858730672392994, Accuracy: 97.555%
Epoch 14/15, Loss: 0.0694512641039677, Accuracy: 97.4525%
Epoch 15/15, Loss: 0.06424320553466678, Accuracy: 97.5675%


In [25]:
# Đánh giá mô hình trên tập test
model.eval()
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        # Di chuyển inputs và labels sang cùng device với model
        inputs = inputs.to(device)  # Di chuyển inputs sang GPU nếu có
        labels = labels.to(device)  # Di chuyển labels sang GPU nếu có
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

test_accuracy = 100 * correct_predictions / total_predictions
print(f"Test Accuracy: {test_accuracy}%")

Test Accuracy: 86.34%
